In [1]:
import matplotlib.pyplot as plt
import itertools
import time
import numpy as np
from tqdm import tqdm
import json
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from numpy.typing import NDArray
from scipy.optimize import minimize
from typing import List, Tuple
import pandas as pd
from sklearn.metrics import r2_score

In [2]:
config = {
    "M_list": [5, 10],
    "delta_list": [0.6, 0.8, 1.0],
    "N": 500,
    "K": 5,
    "B": 100,
    "r_mean": 0.8,
    "r_std": 0.1,
    "r_min": 0.5,
    "r_max": 1.1
}

param_combos = list(itertools.product(config["M_list"], config["delta_list"]))

In [3]:
param_combos

[(5, 0.6), (5, 0.8), (5, 1.0), (10, 0.6), (10, 0.8), (10, 1.0)]

In [3]:
# 各手法のパラメータ定義
quantiles = [0.95, 0.90, 0.85, 0.80]
boot_k = {99: 2.576, 95: 1.96, 90: 1.645}
penalties = {'ebpa3': 0.30, 'ebpa4': 0.40, 'ebpa5': 0.50,'ebpa6': 0.60}

test = {}
for M, delta in param_combos:
    key = f"M{M}_delta{delta}"
    test[key] = {}
    methods = ['so', 'po']
    methods += [f'quan{int(q*100)}' for q in quantiles]
    methods += [f'boot{p}' for p in boot_k]
    methods += list(penalties.keys())
    for m in methods:
        test[key][m] = {
            'sales_ratio': [],
            'true_sales_ratio': [],
            'range': [],
            'range_per_product': [],
            'prices': [],
            'time': []
        }
    test[key]['r2_list'] = []

In [5]:
test.keys()

dict_keys(['M5_delta0.6', 'M5_delta0.8', 'M5_delta1.0', 'M10_delta0.6', 'M10_delta0.8', 'M10_delta1.0'])

In [10]:
test["M5_delta0.6"]["ebpa3"]["sales_ratio"]

[]